## Python for Finance 4 - Low Risk Investments

In [1]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd
from pandas_datareader import data as web # Reads stock data 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

import datetime as dt # For defining dates
import mplfinance as mpf # Matplotlib finance

import time

# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

#### Dates & Other Constants

In [2]:
# Define path to files
PATH = "../data/stock-list/"

# Start date defaults
ST_YEAR = 2017
ST_MONTH = 1
ST_DAY = 1
ST_DATE_STR = f"{ST_YEAR}-{ST_MONTH}-{ST_DAY}"
ST_DATE_DATETIME = dt.datetime(ST_YEAR, ST_MONTH, ST_DAY)

# End date defaults
EN_YEAR = 2021
EN_MONTH = 12
EN_DAY = 31
EN_DATE_STR = f"{EN_YEAR}-{EN_MONTH}-{EN_DAY}"
EN_DATE_DATETIME = dt.datetime(EN_YEAR, EN_MONTH, EN_DAY)

#### Get Stock File Names in a List

In [3]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers

['A',
 'AA',
 'AAL',
 'AAME',
 'AAN',
 'AAOI',
 'AAON',
 'AAP',
 'AAPL',
 'AAT',
 'AAWW',
 'ABBV',
 'ABC',
 'ABCB',
 'ABEO',
 'ABG',
 'ABIO',
 'ABM',
 'ABMD',
 'ABR',
 'ABT',
 'ABTX',
 'AC',
 'ACA',
 'ACAD',
 'ACBI',
 'ACC',
 'ACCO',
 'ACER',
 'ACGL',
 'ACHC',
 'ACHV',
 'ACIW',
 'ACLS',
 'ACM',
 'ACMR',
 'ACN',
 'ACNB',
 'ACOR',
 'ACRE',
 'ACRS',
 'ACRX',
 'ACTG',
 'ACU',
 'ADBE',
 'ADC',
 'ADES',
 'ADI',
 'ADM',
 'ADMA',
 'ADMP',
 'ADNT',
 'ADP',
 'ADSK',
 'ADT',
 'ADTN',
 'ADUS',
 'ADVM',
 'ADXS',
 'AE',
 'AEE',
 'AEHR',
 'AEIS',
 'AEL',
 'AEMD',
 'AEO',
 'AEP',
 'AERI',
 'AES',
 'AEY',
 'AFG',
 'AFI',
 'AFL',
 'AGCO',
 'AGE',
 'AGEN',
 'AGFS',
 'AGIO',
 'AGLE',
 'AGM',
 'AGNC',
 'AGO',
 'AGR',
 'AGRX',
 'AGS',
 'AGTC',
 'AGX',
 'AGYS',
 'AHH',
 'AHT',
 'AIG',
 'AIMC',
 'AIN',
 'AINC',
 'AIR',
 'AIRG',
 'AIRI',
 'AIRT',
 'AIT',
 'AIV',
 'AIZ',
 'AJG',
 'AJRD',
 'AJX',
 'AKAM',
 'AKBA',
 'AKR',
 'AKTS',
 'AL',
 'ALB',
 'ALBO',
 'ALCO',
 'ALDX',
 'ALE',
 'ALEC',
 'ALEX',
 'ALG',
 'ALGN

#### Function that Returns a Dataframe from a CSV

In [4]:
def get_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv')
    except FileNotFoundError:
        pass
        # print("File Doesn't Exist")
    else:
        return df

#### Function that Saves Dataframe to CSV

In [5]:
def save_dataframe_to_csv(df, ticker):
    df.to_csv(PATH + ticker + '.csv')

#### Return 1st Valid Date in Dataframe

In [6]:
def get_valid_dates(df, sdate, edate):  
    try:
        mask = (df['Date'] > sdate) & (df['Date'] <= edate) 
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
        sm_date = sm_df.index.min()
        last_date = sm_df.index.max()
        date_leading = '-'.join(('0' if len(x)<2 else '')+x for x in sm_date.split('-'))
        date_ending = '-'.join(('0' if len(x)<2 else '')+x for x in last_date.split('-'))
    except Exception:
        print("Date Corrupted")
    else:
        return date_leading, date_ending

#### Returns Return on Investment over Time

In [7]:
def roi_between_dates(df, sdate, edate):
    try: 
        start_val = df.loc[sdate,'Adj Close'] 
        end_val = df.loc[edate,'Adj Close']
        roi = ((end_val - start_val) / start_val)
    except Exception:
        print("Data Corrupted")
    else:
        return roi

#### Get Mean Between Dates

In [8]:
def get_mean_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].mean()

#### Get Standard Deviation Between Dates

In [9]:
def get_sd_between_dates(df, sdate, edate):
    mask = (df['Date'] > sdate) & (df['Date'] <= edate)
    return df.loc[mask]["Adj Close"].std()

#### Get Coefficient of Variation Between Dates

In [10]:
def get_cov_between_dates(df, sdate, edate):
    mean = get_mean_between_dates(df, sdate, edate)
    sd = get_sd_between_dates(df, sdate, edate)
    return sd / mean

#### Merge Multiple Stocks in One Dataframe by Column Name

In [11]:
def merge_df_by_column_name(col_name, sdate, edate, *tickers):
    # Will hold data for all dataframes with the same column name
    mult_df = pd.DataFrame()
    
    for x in tickers:
        df = get_df_from_csv(x)
        df['Date'] = pd.to_datetime(df['Date'])
        # Use a mask to grab data between defined dates
        mask = (df['Date'] >= sdate) & (df['Date'] <= edate)
        mult_df[x] = df.loc[mask][col_name]
        
    return mult_df

---
NEW


#### 11 Stock Market Sectors

**Technology**: Manufacturing of electronics, software, or related to information technology

**Health Care**: Biotech, hospitals, medical devices, drugs

**Financials**: Banks, investment funds, and insurance

**Real Estate**: Residential, industrial, and retail real estate

**Energy**: Production and supply of energy

**Materials**: Mining, refining, chemical, and forestry

**Consumer Discretionary**: Retailers, apparel, media, durables and services

**Industrials**: Construction, machinery, fabrication, manufacturing, defense, and aerospace

**Utilities**: Direct providers of electric, gas, and water

**Consumer Staples**: Food, beverage as well as products consumers deem essential

**Telecommunication**: Cable, internet providers, wireless, and satellite

#### Get Sector Stocks

In [12]:
# For Windows
sec_df = pd.read_csv('../data/stock_sectors.csv')

# Get Industrials DF
indus_df = sec_df.loc[sec_df['Sector'] == "Industrials"]
health_df = sec_df.loc[sec_df['Sector'] == "Health Care"]
it_df = sec_df.loc[sec_df['Sector'] == "Information Technology"]
comm_df = sec_df.loc[sec_df['Sector'] == "Communication Services"]
staple_df = sec_df.loc[sec_df['Sector'] == "Consumer Staples"]
discretion_df = sec_df.loc[sec_df['Sector'] == "Consumer Discretionary"]
utility_df = sec_df.loc[sec_df['Sector'] == "Utilities"]
financial_df = sec_df.loc[sec_df['Sector'] == "Financials"]
material_df = sec_df.loc[sec_df['Sector'] == "Materials"]
restate_df = sec_df.loc[sec_df['Sector'] == "Real Estate"]
energy_df = sec_df.loc[sec_df['Sector'] == "Energy"]

In [13]:
sec_df

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care
...,...,...,...
500,YUM,Yum! Brands,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials


#### Returns a DF with ROIs for all Stocks

In [14]:
def get_rois_for_stocks(stock_df):
    # Will hold all tickers & stock rois
    tickers = []
    rois = []

    # iterrows provides the index and column for each row in the DF
    for index, row in stock_df.iterrows():
        df = get_df_from_csv(row['Symbol'])
    
        # If we can't find the ticker delete it from the dataframe
        if df is None:
            pass
            # print(row['Symbol'], " is not available")
        else:
            tickers.append(row['Symbol'])
            sdate, edate = get_valid_dates(df, '2018-01-01', '2020-12-31')
            df = df.set_index(['Date'])
            roi = roi_between_dates(df, sdate, edate)
            rois.append(roi)
    return pd.DataFrame({'Ticker':tickers, 'ROI':rois})

In [15]:
industrial = get_rois_for_stocks(indus_df)
health_care = get_rois_for_stocks(health_df)
it = get_rois_for_stocks(it_df)
commun = get_rois_for_stocks(comm_df)
staple = get_rois_for_stocks(staple_df)
discretion = get_rois_for_stocks(discretion_df)
utility = get_rois_for_stocks(utility_df)
finance = get_rois_for_stocks(financial_df)
material = get_rois_for_stocks(material_df)
restate = get_rois_for_stocks(restate_df)
energy = get_rois_for_stocks(energy_df)

In [16]:
industrial

,Ticker,ROI
0,MMM,-0.183592
1,AOS,-0.060680
2,ALK,-0.273133
3,ALLE,0.504415
4,AAL,-0.694661
...,...,...
60,VRSK,1.195231
61,GWW,0.831677
62,WAB,-0.100293
63,WM,0.452345


#### Top Industrials

In [17]:
industrial.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
24,GNRC,3.734749
10,CPRT,1.918578
55,TDG,1.518984
9,CTAS,1.325593
40,ODFL,1.206289


#### Top Healthcare

In [18]:
health_care.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
24,DXCM,5.375582
32,IDXX,2.167543
58,WST,1.887308
48,RMD,1.575014
15,CTLT,1.475499


#### Top Information Tech

In [19]:
it.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
21,ENPH,66.488464
2,AMD,7.352459
50,PAYC,4.622203
57,NOW,3.178471
69,ZBRA,2.705814


#### Top Communications

In [20]:
commun.sort_values(by=['ROI'], ascending=False).head(20)

,Ticker,ROI
15,NFLX,1.689262
21,TWTR,1.209302
19,TMUS,1.105715
4,CHTR,0.895722
14,LYV,0.722861
20,DIS,0.666390
2,GOOG,0.644958
1,GOOGL,0.633082
10,FB,0.505677
0,ATVI,0.472510


#### Top Staples

In [21]:
staple.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
10,EL,1.145196
9,COST,1.112863
19,MKC,0.977708
3,CHD,0.838448
29,WMT,0.552627


#### Top Discretionary

In [22]:
discretion.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
52,TSLA,10.007861
17,ETSY,7.541047
10,CMG,3.733606
43,POOL,1.931029
51,TGT,1.839621


#### Top Utilities

In [23]:
utility.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
0,AES,1.402913
18,NEE,1.136333
4,AWK,0.819224
26,WEC,0.530380
27,XEL,0.519897


#### Top Finance

In [24]:
finance.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
39,MSCI,2.612068
34,MKTX,1.860333
6,AJG,1.112316
37,MCO,1.044882
48,SPGI,1.012123


#### Top Materials

In [25]:
material.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
3,BLL,1.474667
22,SHW,0.824243
14,LIN,0.785525
0,APD,0.771643
17,NEM,0.696606


#### Top Real Estate

In [26]:
restate.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
21,SBAC,0.772695
8,EQIX,0.692929
17,PLD,0.689758
1,AMT,0.685783
5,CCI,0.622156


#### Top Energy

In [27]:
energy.sort_values(by=['ROI'], ascending=False).head(5)

,Ticker,ROI
7,HES,0.168441
13,OKE,-0.131482
8,KMI,-0.146362
2,COP,-0.217487
18,WMB,-0.218709


#### Portfolio List

In [28]:
port_list = ["GNRC", "DXCM", "AMD", "NFLX", "COST", "TGT", "AES", "MSCI", 
             "NEM", "SBAC", "HES"]

#### Check for Correlation

In [29]:
mult_df = merge_df_by_column_name('daily_return',  '2018-01-01', 
                                  '2020-12-31', *port_list)
mult_df

# Generate a Correlation Matrix
mult_df.corr()

,GNRC,DXCM,AMD,NFLX,COST,TGT,AES,MSCI,NEM,SBAC,HES
GNRC,1.000000,0.289526,0.395187,0.340732,0.465304,0.338411,0.433323,0.515743,0.250840,0.444762,0.426870
DXCM,0.289526,1.000000,0.328297,0.307643,0.265366,0.196806,0.218424,0.469509,0.147613,0.243462,0.243240
AMD,0.395187,0.328297,1.000000,0.459581,0.394315,0.340573,0.258975,0.485819,0.191451,0.294906,0.288337
NFLX,0.340732,0.307643,0.459581,1.000000,0.430674,0.259971,0.204158,0.449132,0.103912,0.216935,0.217857
COST,0.465304,0.265366,0.394315,0.430674,1.000000,0.500433,0.319885,0.482322,0.183703,0.455022,0.234526
TGT,0.338411,0.196806,0.340573,0.259971,0.500433,1.000000,0.305004,0.325772,0.092807,0.304364,0.245106
AES,0.433323,0.218424,0.258975,0.204158,0.319885,0.305004,1.000000,0.483399,0.176952,0.333249,0.470413
MSCI,0.515743,0.469509,0.485819,0.449132,0.482322,0.325772,0.483399,1.000000,0.258857,0.498281,0.411647
NEM,0.250840,0.147613,0.191451,0.103912,0.183703,0.092807,0.176952,0.258857,1.000000,0.275892,0.149227
SBAC,0.444762,0.243462,0.294906,0.216935,0.455022,0.304364,0.333249,0.498281,0.275892,1.000000,0.273548


In [30]:
# Get the number of samples
days = len(mult_df.index)
days

756

In [31]:
def get_stock_price_on_date(ticker, date):
    df = get_df_from_csv(ticker)
    df = df.set_index(['Date'])
    return df.loc[date,'Adj Close']

In [32]:
# 01-04-2021 is the 1st date in 2021
print("GNRC $", get_stock_price_on_date('GNRC', '2021-01-04')) # 2 449.12
print("DXCM $", get_stock_price_on_date('DXCM', '2021-01-04')) # 1 357.74
print("AMD $", get_stock_price_on_date('AMD', '2021-01-04')) # 4 369.2
print("NFLX $", get_stock_price_on_date('NFLX', '2021-01-04')) # 1 522.86
print("COST $", get_stock_price_on_date('COST', '2021-01-04')) # 1 377.88
print("TGT $", get_stock_price_on_date('TGT', '2021-01-04')) # 3 527.46
print("AES $", get_stock_price_on_date('AES', '2021-01-04')) # 18 410.76
print("MSCI $", get_stock_price_on_date('MSCI', '2021-01-04')) # 1 435.40
print("NEM $", get_stock_price_on_date('NEM', '2021-01-04')) # 7 434.49
print("SBAC $", get_stock_price_on_date('SBAC', '2021-01-04')) # 2 548.84
print("HES $", get_stock_price_on_date('HES', '2021-01-04')) # 9 472.23

# Total Investment
tot_inv = 449.12 + 357.74 + 369.2 + 522.86 + 377.88 + 527.46 + 410.76 + \
435.40 + 434.49 + 548.84 + 472.23
tot_inv

GNRC $ 224.5599975585937
DXCM $ 357.739990234375
AMD $ 92.3000030517578
NFLX $ 522.8599853515625
COST $ 377.8815307617188
TGT $ 174.4677734375
AES $ 22.35209846496582
MSCI $ 433.8714599609375
NEM $ 60.88787078857422
SBAC $ 274.4242248535156
HES $ 51.91611862182617


4905.980000000001

#### Returns Portfolio Weight

In [33]:
def get_port_weight(price, total):
    return price / total

In [34]:
gnrc_w = get_port_weight(449.12, 4905.98)
dxcm_w = get_port_weight(357.74, 4905.98)
amd_w = get_port_weight(369.2, 4905.98)
nflx_w = get_port_weight(522.86, 4905.98)
cost_w = get_port_weight(377.88, 4905.98)
tgt_w = get_port_weight(527.46, 4905.98)
aes_w = get_port_weight(410.76, 4905.98)
msci_w = get_port_weight(435.40, 4905.98)
nem_w = get_port_weight(434.49, 4905.98)
sbac_w = get_port_weight(548.84, 4905.98)
hes_w = get_port_weight(472.23, 4905.98)

wts = np.array([gnrc_w, dxcm_w, amd_w, nflx_w, cost_w, tgt_w, aes_w, msci_w, \
                nem_w, sbac_w, hes_w])
wts

port_var = np.dot(wts.T, np.dot(mult_df.cov() * 756, wts))
print("Portfolio Covariance :", port_var)
print("GNRC Var :", mult_df["GNRC"].var() * 756)
print("NFLX Var :", mult_df["NFLX"].var() * 756)

Portfolio Covariance : 0.182210897961409
GNRC Var : 0.43946543607242516
NFLX Var : 0.5477253623043382


#### Different Types of Risk

There is risk that you can limit through diversifaction (Idiosyncratic) and risk that you can't (Systematic). Systematic risk is caused by unforeseen conditions such as wars, recessions, natural disasters, dramatic changes in consumer actions, etc.

Data tells us that if we make a portfolio made up of approximately 25 stocks that aren't correlated that we can dramatically lower idiosyncratic risk. That is one reason why people invest in indexes. You can further lower risk by investing in other countries, bonds and cash.

#### Diversifiable Risk

In [35]:
# Diversifiable Risk = Portfolio Variance - All Weighted Variances
def calc_diversifiable_risk(df, tickers, weights):
    # Gets number of days
    days = len(mult_df.index)
    # Calculate covariance of portfolio
    port_covar = np.dot(weights.T, np.dot(df.cov() * days, weights)) 
    
    i = 0
    while i < len(tickers):
        wt_sq = weights[i] ** 2
        stk_var = mult_df[tickers[i]].var() * days
        wt_var = wt_sq * stk_var
        port_covar = port_covar - wt_var
        i += 1
    return port_covar

In [36]:
div_risk = calc_diversifiable_risk(mult_df, port_list, wts)
div_risk

0.13626890697904706

#### Find Risk that Can't be Diversified

In [37]:
print("Systematic Risk :", (port_var - div_risk))

Systematic Risk : 0.04594199098236193
